<a href="https://colab.research.google.com/github/GabiSnow/EvolutiveNeuralNetwork/blob/main/TFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Includes**



In [ ]:
from operator import truediv
from wsgiref.validate import validator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import copy

**Variables globales**

In [ ]:
# Inicial de cada Palo, Terbol, Picas, Corazones y Diamantes
nombrePalos = ["T","P","C","D"]

# Tamaño del tablero
NumColums = 10
NumFilas = 28

# Para controlar los pesos y el limite de la población inicial
# numero de neuronas en capa oculta y capa oculta
nEntradas = NumColums*NumFilas
nOcultas = int((2/3) * nEntradas)

# Tamaño de la poblacion a generar
nIndividuos = 90

# factor Nguyen Widrow
# el limite de los valores para los sesgos esta comprendido entre -B y +B
factorB = pow((0.7 * nOcultas),( 1/nEntradas))

# Valores de los operadores geneticos
k = 8 # Tamaño del torneo
cProb = 0.65 # Probabilidad de cruce
mProb = 0.55 # Probabilidad de mutacion

# Numero de veces que se ejecuta el algoritmo evolutivo para una poblacion de un mismo tablero
nIteraEvol = 16

# Numero de tableros sobre los que trabajaremos
numTabl = 10

**Declaración de la clase Carta**

In [ ]:
class carta ():
  valor = 0
  oculta = True
  palo = 0

  def __init__(self,val,ocul,pal):
      self.valor = val
      self.oculta = ocul
      self.palo = pal

  def setPos(self,ocul):
      self.oculta = ocul

  def __repr__(self):

    if self.valor == 0:
      return "   "

    if self.oculta == True:
      return "XXX"

    v = str(self.valor)
    if self.valor < 10:
      v = v + " "
    return nombrePalos[self.palo] + v

# valor:
# 1 - 10 -> cartas del 1 al  10
# 11 -> J
# 12 -> Q
# 13 -> K

# val -> valor numerico de la carta
# ocul -> true si la carta esta oculta
# pal -> palo de la carta , van del 0 al 3

**Declaración de la clase Tablero**

Clase con una matriz que representa el tablero de juego, tambien tenemos el array UltimaCarta que nos dice cuantas cartas hay en cada fila, y barajaRest que guarda la baraja restante

In [ ]:
class tableroClass ():
  tablero = []
  tableroRed = [] # tablero adaptado como entrada de la red neuronal
  ultimaCarta = []
  barajaRest = []

  def __init__(self,tabler,ult,barajaR):
      self.tablero = tabler
      self.ultimaCarta = ult
      self.barajaRest = barajaR

      tam = self.tablero.shape

      fil = tam[0]
      col = tam[1]

      self.tableroRed = []
      for i in range (fil):
        for j in range(col):
          self.tableroRed.append(self.tablero[i,j].valor + 100*self.tablero[i,j].palo)

  def __repr__(self):
      return "\n-------------------------------\n" + str(self.tablero) + "\n\n" + str(self.ultimaCarta)

**Genera un tablero aleatorio**

In [ ]:
barajaGeneral = []
# Creo una bajara unica ordenada que contendria las 2 barajas de 52 cartas (4 palos)

# bucle por palo
for paloA in range(4):
  # bucle por numero de carta
  for cartaA in range(13):
    # 2 veces porque hay 2 cartas de cada tipo
    barajaGeneral.append(carta(cartaA + 1,True,paloA))
    barajaGeneral.append(carta(cartaA + 1,True,paloA))

In [ ]:
cartNula = carta(0,False,-1)

def generaTableroAle():

  nuevaBaraja = []
  nuevaBaraja[:] = barajaGeneral

  # Barajamos la baraja
  for i in range(150):

    # Se eligen dos cartas aleatorias y se intercambian entre si
    car1 = random.randint(0,103)
    car2 = random.randint(0,103)

    aux = nuevaBaraja[car1]
    nuevaBaraja[car1] = nuevaBaraja[car2]
    nuevaBaraja[car2] = aux

  tableroN = np.full((NumFilas,NumColums),cartNula)
  for i in range(4):
    for j in range(10):
      tableroN[i][j] = copy.copy(nuevaBaraja.pop())

  for j in range(4):
    tableroN[4][j] = copy.copy(nuevaBaraja.pop())

  for j in range(6):
    cartAux = copy.copy(nuevaBaraja.pop())
    cartAux.oculta = False
    tableroN[4][j+4] = cartAux

  for j in range(4):
    cartAux = copy.copy(nuevaBaraja.pop())
    cartAux.oculta = False
    tableroN[5][j] = cartAux

  ultimaC = [5,5,5,5,4,4,4,4,4,4]

  return tableroClass(tableroN,ultimaC,nuevaBaraja)

**Lista de las opciones posibles**

Genera todos los posibles movimientos a partir del estado actual de un determinado tablero


In [ ]:
def opciones(tab):

  tablero = tab.tablero
  ultimaCarta = tab.ultimaCarta
  baraja = tab.barajaRest
  # A partir de la ultima carta o un conjunto de cartas de una columna
  # se comprueba si se puede mover a alguna otra columna

  # lista de posibles tableros

  solucion = []

  # bucle que recorre cada columna
  for i in range(NumColums-1,-1,-1):

      posCarta = ultimaCarta[i]

      # para evitar columnas vacias
      if posCarta != -1:

          #cartaAnterior = tablero[posCarta,i].valor -1
          conjunto = []

          # Recorro todas las cartas de la columna (mientras esten en orden)
          for cart in range(posCarta,-1,-1):

              # la carta que estamos mirando
              cartaActual = tablero[cart,i]

              # compruebo si se puede añadir la carta al conjunto
              # para ello debe ser consecutiva, del mismo palo y no estar oculta
              if len(conjunto) == 0:
                conjunto.append(cartaActual)
              else:
                  if (conjunto[-1].valor == cartaActual.valor - 1
                      and conjunto[-1].palo == cartaActual.palo and cartaActual.oculta == False):
                    conjunto.append(cartaActual)
                  else:
                    break

              # se comprueba a que posiciones nuevas se puede mover la carta / el conjunto
              for j in range(NumColums):
                  if j != i:

                      # valor que deberia tener la carta de otra columna para poder moverla
                      valorCartaSig = cartaActual.valor + 1

                      # guardamos la siguiente carta a mirar
                      sig = tablero[ultimaCarta[j],j]


                      if sig.valor == valorCartaSig or ultimaCarta[j] == -1:

                          # generamos el nuevo tablero y la nueva lista de ultimas cartas

                          proxTablero = np.empty_like(tablero)
                          proxTablero[:] = tablero

                          proxUltimaCarta = []
                          proxUltimaCarta[:] = ultimaCarta

                          proxBaraja = []
                          proxBaraja[:] = baraja

                          numCartasConjunto = np.size(conjunto)
                          pos = ultimaCarta[j]

                          proxTablero[pos+1:pos+1+numCartasConjunto,j] = list(reversed(conjunto))

                          proxTablero[cart:cart+numCartasConjunto,i] = cartNula

                          # La carta anterior pasa a dejar de estar oculta (en caso de que lo estuviese)
                          if cart-1 >= 0:
                            proxTablero[cart-1,i] = copy.copy(proxTablero[cart-1,i])
                            proxTablero[cart-1,i].oculta = False

                          proxUltimaCarta[i] = proxUltimaCarta[i] - numCartasConjunto
                          proxUltimaCarta[j] = proxUltimaCarta[j] + numCartasConjunto

                          solucion.append(tableroClass(proxTablero,proxUltimaCarta,proxBaraja))

              # para considerar el anterior en la siguiente iteración
              #cartaAnterior = cartaActual

  return solucion

#tabler = creaTablero()
#print(tabler)
#opciones(tabler)

**Declaracion del Fitness**

El fitness en este caso es bastante simple, se puntua cuando dos cartas correlativas tienen un valor consecutivo, y se recibe una puntuacion extra si el palo de ambas cartas es el mismo, esto hace que se valore mucho mas un conjunto de cartas consecutivas donde todas sean del mismo palo a un conjunto con distintos palos.

In [ ]:
def fitness(tab):

  puntuacion = 0
  tablero = tab.tablero
  ultimaCarta = tab.ultimaCarta

  for i in range(NumColums-1,-1,-1):

      posCarta = ultimaCarta[i]

      # para evitar columnas vacias
      if posCarta != -1:

          conjunto = []

          # la variable Incremento hace que se le de mas valor
          #a que dos cartas consecutivas sean del mismo palo
          incremento = 0.5

          # Recorro todas las cartas de la columna (mientras esten en orden)
          for cart in range(posCarta,-1,-1):

              # la carta que estamos mirando
              cartaActual = tablero[cart,i]

              # compruebo si se puede añadir la carta al conjunto
              # para ello debe ser consecutiva, del mismo palo y no estar oculta
              if len(conjunto) == 0:
                conjunto.append(cartaActual)
              else:
                  if (conjunto[-1].valor == cartaActual.valor - 1 and cartaActual.oculta == False):
                    puntuacion += 0.5
                    conjunto.append(cartaActual)

                    if (conjunto[-1].palo == cartaActual.palo):
                      puntuacion += incremento
                      incremento += 0.4 # esto se puede cambiar para regular el crecimiento
                  else:
                    break

  return puntuacion/10

**Algoritmo Evolutivo**

**Declaracion de la clase Red Neuronal**

Se define la clase Red Neuronal, que guardara los distintos sesgos y pesos de una determinada red, añadiendo ademas metodos para evaluar dicha red ante una determinada entrada.

La clase Nodo solo ha sido creada para una mejor legibilidad del codigo.

In [ ]:
# Funcion Sigmoide
def sigmoide(x):
    sig = 1 / (1 + math.exp(-x))
    return sig

# Para que cada nodo de la capa de entrada tenga una lista de pesos de salida asociada
class nodo ():
  pesos = []

  def __init__(self,pes):
      self.pesos = pes

class redNeuronal():

  sesgoEn = []
  sesgoOc = 0
  nodosEntrada = []

  pesosEntrada = []
  pesosOculta = []

  #salida = -1

  def __init__(self,sesgEn,pesosEntrad,sesgOc,pesosOcult):

    self.sesgoEn = sesgEn
    self.sesgoOc = sesgOc
    self.pesosOculta = pesosOcult
    self.pesosEntrada = pesosEntrad

    # guardamos los datos de los pesos de entrada en los distintos nodos de la capa de entrada
    cont = 0;
    for i in range(nEntradas ):
      pes = []
      for j in range(nOcultas):
        pes.append(pesosEntrad[cont])
        cont = cont + 1

      self.nodosEntrada.append(nodo(pes))

  # evalua una lista de entrada
  def salida(self,entrada):

    # estas salidas de la capa de entrada son las entradas de la capa oculta
    salidasEn = []
    for i in range(nOcultas):
      sal = self.sesgoEn[i]
      for j in range(nEntradas):
        sal += self.nodosEntrada[j].pesos[i] * entrada[j]

      salidasEn.append(sigmoide(sal))

    # habria que aplicarle aqui a la lisa de salidas la funcion sigmoide

    #print(salidasEn)
    sal = self.sesgoOc

    for i in range(nOcultas):
      sal += self.pesosOculta[i] * salidasEn[i]

    # en este caso no aplico la funcion sigmoide como salida
    #res = sigmoide(sal)
    self.puntuacion = sal #res

    return sal #res

  def setPesos(self,pesosEntrad,pesosOcult):

    self.pesosEntrada = pesosEntrad
    self.pesosOculta = pesosOcult
    self.nodosEntrada = []
    cont = 0;
    for i in range(nEntradas ):
      pes = []
      for j in range(nOcultas):
        pes.append(pesosEntrad[cont])
        cont = cont + 1

      self.nodosEntrada.append(nodo(pes))

  def __repr__(self):
    return ("\nRed Neuronal:\nSesgo C.Entrada:\n"+str(self.sesgoEn)+"\nPesos C.Entrada:\n" +str(self.pesosEntrada)
    +"\nSesgo C.Oculta:\n"+str(self.sesgoOc)+"\nPesos C.Oculta:\n"+ str(self.pesosOculta)+"\n--------------\n")

**Inicializacion Red Neuronal Evolutiva, generación de la población inicial**

In [ ]:
# longitud de los pesos
def calculaV(lista):
  return math.sqrt(sum([number**2 for number in lista]))

def actualizaPesos(lista,factorB):
  valorV = calculaV(lista)
  return [(number*factorB)/valorV  for number in lista]

In [ ]:
# Genera la lista de individuos iniciales
def generaPoblacion():
  poblacion = []
  for i in range(nIndividuos):
    # Genero el sesgo
    sesgoE = []
    sesgoO = random.uniform(-factorB,factorB)

    #Genero los pesos
    pesosE = []
    pesosO = []

    # Inicializa los pesos
    for j in range(nEntradas*nOcultas):
      pesosE.append(random.uniform(-0.2,0.2))
    for j in range(nOcultas):
      sesgoE.append(random.uniform(-factorB,factorB))
      pesosO.append(random.uniform(-0.2,0.2))

    # Actualiza el valor de los pesos

    pesosE = actualizaPesos(pesosE,factorB)
    pesosO = actualizaPesos(pesosO,factorB)

    poblacion.append([redNeuronal(sesgoE,pesosE,sesgoO,pesosO),-1])

  return poblacion


**Cruce y mutación**

El cruce se aplicara de forma independiente para los pesos de la capa de entrada, los de la capa oculta y los sesgos.

El cruce se hará dividiendo cada capa de cada padre en 3 partes, mezclandolas posteriormente con las del otro padre para generar 2 hijos.


In [ ]:

# divide una lista en tres
def dividir(lista):
    tercio = round(len(lista)/3)
    return lista[:tercio],lista[tercio:2*tercio], lista[2*tercio:]


def mutacionAux(m1): #Enteros(m1):

    rn = random.randint(0,len(m1)-1)

    mutacion = random.randint(0,1)

    # He elegido 0.2 pero puede cambiarse por un valor porcentual
    if mutacion == 0:
      mutacion = -0.2
    else:
      mutacion = 0.2

    m1[rn]+=mutacion

    return m1

def mutacion(red):
  # mutacion de la capa de entrada
  pEn = mutacionAux(red.pesosEntrada)
  pOc = mutacionAux(red.pesosOculta)

  red.sesgoEn = mutacionAux(red.sesgoEn)

  red.setPesos(pEn,pOc)
  return red

def cruceInde(gen1,gen2,muta = False):

  # capa de entrada
  mitad1_Gen1, mitad2_Gen1, mitad3_Gen1 = dividir(gen1.pesosEntrada)
  mitad1_Gen2, mitad2_Gen2, mitad3_Gen2= dividir(gen2.pesosEntrada)

  capEntrada1 = mitad1_Gen1 + mitad3_Gen1 + mitad2_Gen2
  capEntrada2 = mitad3_Gen2 + mitad2_Gen1 + mitad1_Gen2

  # capa oculta
  mitad1_Gen1, mitad2_Gen1, mitad3_Gen1 = dividir(gen1.pesosOculta)
  mitad1_Gen2, mitad2_Gen2, mitad3_Gen2 = dividir(gen2.pesosOculta)

  capOculta1 = mitad2_Gen2 + mitad1_Gen1 + mitad3_Gen1
  capOculta2 = mitad3_Gen2 + mitad2_Gen1 + mitad1_Gen2

  # cruzamos los sesgos
  sesd1_1,sesd1_2,sesd1_3 = dividir(gen1.sesgoEn)
  sesd2_1,sesd2_2,sesd2_3 = dividir(gen2.sesgoEn)

  sesgoEn1 = sesd2_2 + sesd1_1 + sesd1_3
  sesgoOc1 = gen1.sesgoOc

  sesgoEn2 = sesd2_1 +sesd2_3 +sesd1_2
  sesgoOc2 = gen2.sesgoOc

  return redNeuronal(sesgoEn1,capEntrada1,sesgoOc1,capOculta1), redNeuronal(sesgoEn2,capEntrada2,sesgoOc2,capOculta2)


#in1 = redNeuronal([1,1],[1,2,3,4,0,0],1,[10,11])
#in2 = redNeuronal([2,2],[5,6,7,8,9,9],2,[20,21])

#in3,in4 = cruceInde(in1,in2)


**Torneo**

Genera un torneo entre "k" elementos de la poblacion y devuelve el mejor de ellos

In [ ]:
def torneo(poblacion,k):

    index_candidatos = []
    candidatos = []


    for i in range(k):

        index = random.randint(1,len(poblacion)-1)

        candidatos.append(poblacion[index][1])
        index_candidatos.append(index)

    mejor = max(candidatos)
    pos = candidatos.index(mejor)

    return poblacion[[index_candidatos[pos]][0]]

**Evaluacion de poblacion**

Se pasa por parametros la poblacion a evaluar, el tablero actual y el fitness de dicho tablero, y se devuelve como puntuacion el error absoluto (a menor puntuacion mejor)

In [ ]:
def evaluarPoblacion(poblacion,tab,eval):

  for i in range(nIndividuos):
    poblacion[i][1] = abs(poblacion[i][0].salida(tab.tableroRed)-eval)

  return poblacion


**Operadores Geneticos**

Genero una nueva poblacion del doble de tamaño de la original, y me quedo con los n mejores (donde n es el tamaño de la poblacion original). El hecho de coger los n mejores hace que estemos aplicando un tipo de elitismo.

In [ ]:
def aplicarOperadoresGeneticos(poblacion, k, cProb, mProb,tab,eval):

    npoblacion = []
    tamPobla = len(poblacion)
    doblePobla = 2* tamPobla

    while len(npoblacion) != doblePobla:
        #Seleccionar padres mediante torneo tamaño k
        padrea = torneo(poblacion,k);
        padreb = torneo(poblacion,k);

        s = [padrea[0],padreb[0]]

        #Cruzar padres con probabilidad cProb
        if random.randint(1,100) <= cProb*100:
            s[0], s[1] = cruceInde(padrea[0],padreb[0])

        #Mutar padres con probabilidad mProb
        for i in range(2):
          if random.randint(1,100) <= mProb*100:
            s[i] = mutacion(s[i])

         #añadir los hijos a la nueva poblacion
        for i in range(2):
            if len(npoblacion) < doblePobla:
                npoblacion.append([s[i],-1])

    # evaluamos la nueva poblacion
    npoblacion = evaluarPoblacion(poblacion,tab,eval)

    # Ordenamos la poblacion dependiendo de la evaluacion (una puntuacion menor es mejor, ya que es el error)
    sortPoblacion = sorted(npoblacion, key=lambda red : red[1]) #reverse = True)

    # Cogemos los n mejores, donde n es el tamaño de la poblacion inicial
    npoblacion = sortPoblacion[:tamPobla]

    return npoblacion #Devolver la nueva poblacion evaluada


**Programa principal, busqueda del mejor modelo**

**Busqueda del mejor movimiento**

A partir de un estado del tablero se generan todas las opciones posibles y se evaluan por el fitness, decidiendo cual es el mejor movimiento.

Tambien se implementara el mismo codigo pero utilizando la red neuronal en vez del fitness.

In [ ]:
def mejorMovimientoFitness(tableroC):

  listaOpcs = opciones(tableroC)

  opcionesEval = []

  nIt = len(listaOpcs)

  # calculo la puntuacion del estado actual, el siguiente estado debe ser mejor
  punt = fitness(tableroC)

  for i in range(nIt):

    nPunt = fitness(listaOpcs[i])

    if nPunt > punt :
      opcionesEval.append([listaOpcs[i],nPunt])

  sortOpcs = sorted(opcionesEval, key=lambda red : red[1], reverse = True)

  if (len(opcionesEval) == 0):
    return -1
  return sortOpcs[0][0]


In [ ]:
def mejorMovimientoRed(redNeuro,tablero):

  listaOpcs = opciones(tablero)

  opcionesEval = []

  nIt = len(listaOpcs)

  # calculo la puntuacion del estado actual, el siguiente estado debe ser mejor
  punt = redNeuro.salida(tablero.tableroRed)

  for i in range(nIt):

    nPunt = redNeuro.salida(listaOpcs[i].tableroRed)

    if nPunt > punt :
      opcionesEval.append([listaOpcs[i],nPunt])

  sortOpcs = sorted(opcionesEval, key=lambda red : red[1], reverse = True)

  if (len(opcionesEval) == 0):
    return -1
  return sortOpcs[0][0]


**Desarrollo del tablero**

Metodos para desarrollar el tablero tanto con el fitness como con el mejor modelo

In [ ]:
# Desarrollamos el tablero N veces
def desarrollarTableroFitness(tabler,N,mostrarParada):

  for i in range(N):
    auxT = mejorMovimientoFitness(tabler)

    # En caso de que no haya un movimiento que mejore el tablero
    if auxT == -1:

      # Si siguen quedando cartas en la baraja tendriamos que poner
      # una nueva carta en cada columna del tablero
      barajaA = []
      barajaA[:] = tabler.barajaRest

      if len(barajaA) == 0:
        if mostrarParada:
          print("Parada: " , i)
        break # no se podria hacer nada mas

      else:
        tableroA = copy.copy(tabler.tablero)

        ultCart = []
        ultCart[:] = tabler.ultimaCarta

        # Se reparten 10 cartas de la baraja
        for j in range(NumColums):
          cartaAux = copy.copy(barajaA.pop())
          cartaAux.oculta = False
          tableroA[ultCart[j] + 1][j] = cartaAux
          ultCart[j] += 1

          tabler = tableroClass(tableroA,ultCart,barajaA)

    else:
      tabler = auxT

  return tabler

In [ ]:
# Desarrollamos el tablero N veces
def desarrollarTableroRedNeuronal(redNeuro,tabler,N,mostrarParada):

  for i in range(N):
    auxT = mejorMovimientoRed(redNeuro,tabler)

    # En caso de que no haya un movimiento que mejore el tablero
    if auxT == -1:

      # Si siguen quedando cartas en la baraja tendriamos que poner
      # una nueva carta en cada columna del tablero
      barajaA = []
      barajaA[:] = tabler.barajaRest

      if len(barajaA) == 0:
        if mostrarParada:
          print("Parada: " , i)
        break # no se podria hacer nada mas

      else:
        tableroA = copy.copy(tabler.tablero)

        ultCart = []
        ultCart[:] = tabler.ultimaCarta

        # Se reparten 10 cartas de la baraja
        for j in range(NumColums):
          cartaAux = copy.copy(barajaA.pop())
          cartaAux.oculta = False
          tableroA[ultCart[j] + 1][j] = cartaAux
          ultCart[j] += 1

          tabler = tableroClass(tableroA,ultCart,barajaA)

    else:
      tabler = auxT

  return tabler

**Busqueda de la mejor red neuronal aplicando el algoritmo genético**

Se define "algoGeneti", metodo que devuelve una poblacion despues de aplicarle n veces el algoritmo genetico

Posteriormete se genera la poblacion inicial y se va actualizando utilizando el metodo algoGeneti, cada vez que utilizamos el metodo es con un tablero distinto (utilizando como entradala poblacion generada en la iteracion anterior)

In [ ]:
# A partir de una poblacion inicial aplica el algoritno n veces y devuelve una nueva poblacion
def algoGeneti(poblacion,tablero):
  eval = fitness(tablero)

  # Genera la poblacion inicial y la evalua
  npoblacion = evaluarPoblacion(poblacion,tablero,eval)

  # Numero X de iteraciones aplicando el algoritmo evolutivo
  for i in range(nIteraEvol):
    # Se genera la nueva poblacion a partir de cruzar a la poblacion anterior, esta ordenada por evaluacion
    nPoblacion = aplicarOperadoresGeneticos(npoblacion,k,cProb,mProb,tablero,eval)

  return nPoblacion

**Creacion de los N tableros sobre los que trabajaremos**

In [ ]:
listaTableros = []

for i in range(numTabl):
  tableroA = generaTableroAle()
  tableroA = desarrollarTableroFitness(tableroA,(i*5) + 5,False)
  listaTableros.append(tableroA)

**Aplicacion del algoritmo genetico sobre los tableros**

In [ ]:
# Se aplica el algoritmo genetico para la poblacion utilizando
# en cada iteracion un tablero distinto para la evaluacion
poblacion = generaPoblacion()
numTableros = len(listaTableros)

for i in range(numTableros):
  print(i)
  poblacion = algoGeneti(poblacion,listaTableros[i])

# poblacion[0] -> el mejor modelo
mejorModelo = poblacion[0][0]


0
1
2
3
4
5
6
7
8
9


**Comparacion entre el mejor modelo de red neuronal y el modelo fitness inicial**

In [ ]:
# Generamos un nuevo tablero
tableroP1 = generaTableroAle()
tableroP2 = copy.copy(tableroP1)

print(tableroP1)

tableroF1 = desarrollarTableroFitness(tableroP1,100,True)
print(tableroF1)
print("Fitness: ",fitness(tableroF1) )
print("\n-------------------\n")
tableroF2 = desarrollarTableroRedNeuronal(mejorModelo,tableroP2,100,True)
print(tableroF2)
print("Fitness: ",fitness(tableroF2) )


-------------------------------
[[XXX XXX XXX XXX XXX XXX XXX XXX XXX XXX]
 [XXX XXX XXX XXX XXX XXX XXX XXX XXX XXX]
 [XXX XXX XXX XXX XXX XXX XXX XXX XXX XXX]
 [XXX XXX XXX XXX XXX XXX XXX XXX XXX XXX]
 [XXX XXX XXX XXX D10 D1  C3  D9  P1  P3 ]
 [D10 C3  T11 D6                         ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                                       ]
 [                   